In [1]:
from cameo import load_model

In [2]:
model = load_model("iJR904")

In [3]:
model

<SolverBasedModel MODELID_1461534 at 0x109e06c50>

In [4]:
from cameo.flux_analysis import simulation

In [5]:
help(simulation)

Help on module cameo.flux_analysis.simulation in cameo.flux_analysis:

NAME
    cameo.flux_analysis.simulation

FILE
    /Users/joao/Documents/repos/cameo/cameo/flux_analysis/simulation.py

DESCRIPTION
    Methods for simulating flux distributions.
    Currently implements:
    * fba - Flux Balance Analysis
    * pfba - Parsimonious Flux Balance Analysis
    * lmoma - (Linear) Minimization of Metabolic Adjustment
    * room - Regulatory On/Off Minimization

FUNCTIONS
    fba(model, objective=None, *args, **kwargs)
        Flux Balance Analysis.
        
        Parameters
        ----------
        model: SolverBasedModel
        objective: a valid objective - see SolverBaseModel.objective (optional)
        
        Returns
        -------
        FluxDistributionResult
            Contains the result of the linear solver.
    
    lmoma(model, reference=None, cache={}, volatile=True, *args, **kwargs)
        Linear Minimization Of Metabolic Adjustment.
        
        Parameters
   

Step 1: Simulate the flux distribution of the Wild-Type
----------------------------------------------------

FBA and PFBA can be used to compute flux distributions for the Wild-Type (model as it as been described) as well as knockouts (next session)

In [6]:
%time
fba_result = simulation.fba(model)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [7]:
%time
pfba_result = simulation.pfba(model)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


Setp 2: Simulate knockouts phenotypes
-----------------------------------

Although PFBA and FBA can be used to simulate the effect of knockouts, other methods have been proven more valuable for that task: MOMA and ROOM. In *cameo* we implement a linear version of MOMA.

*******************************************
Simulating knockouts:

* Manipulate the bounds of the reaction (or use the shorthand method knock_out)

In [9]:
model.reactions.PGI

Id,PGI
Stoichiometry,g6p_c <=> f6p_c
Lower bound,-999999.000000
Upper bound,999999.000000


In [10]:
model.reactions.PGI.knock_out()
model.reactions.PGI

Id,PGI
Stoichiometry,g6p_c --> f6p_c
Lower bound,0.000000
Upper bound,0.000000


* Simulate using different methods:

In [11]:
%time
fba_knockout_result = simulation.fba(model)
fba_knockout_result[model.objective]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


0.905983

In [12]:
pfba_knockout_result = simulation.pfba(model)
pfba_knockout_result[model.objective]

0.905983

MOMA and ROOM relly on a reference (wild-type) flux distribution and we can use the one previously computed.

**Parsimonious FBA references seem to produce better results using this methods**

In [26]:
lmoma_result["2 * EX_glc_lp_e_rp_"]

-18.7358

In [14]:
%time
lmoma_result = simulation.lmoma(model, reference=pfba_result.fluxes)
lmoma_result[model.objective]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


0.791393

In [19]:
%time
room_result = simulation.room(model, reference=pfba_result.fluxes)
room_result[model.objective]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


0.887440

In [27]:
room_result